In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl

In [2]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories: ', dataset.num_classes)

Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories:  7


In [4]:
g = dataset[0]

print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([False, False, False,  ..., False, False, False]), 'val_mask': tensor([False, False,  True,  ..., False, False, False]), 'test_mask': tensor([ True,  True, False,  ..., False, False, False]), 'label': tensor([4, 4, 4,  ..., 4, 3, 3]), 'feat': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]])}
Edge features
{'__orig__': tensor([  298,  9199,  1153,  ..., 10415,  5255,  6356])}


In [5]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [6]:
model

GCN(
  (conv1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (conv2): GraphConv(in=16, out=7, normalization=both, activation=None)
)

In [7]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.156 (best 0.156), test acc: 0.141 (best 0.141)
In epoch 5, loss: 1.893, val acc: 0.458 (best 0.458), test acc: 0.442 (best 0.442)
In epoch 10, loss: 1.817, val acc: 0.540 (best 0.540), test acc: 0.548 (best 0.548)
In epoch 15, loss: 1.719, val acc: 0.572 (best 0.572), test acc: 0.582 (best 0.582)
In epoch 20, loss: 1.601, val acc: 0.592 (best 0.592), test acc: 0.615 (best 0.615)
In epoch 25, loss: 1.465, val acc: 0.642 (best 0.642), test acc: 0.673 (best 0.673)
In epoch 30, loss: 1.316, val acc: 0.660 (best 0.660), test acc: 0.696 (best 0.695)
In epoch 35, loss: 1.160, val acc: 0.672 (best 0.674), test acc: 0.715 (best 0.704)
In epoch 40, loss: 1.003, val acc: 0.692 (best 0.692), test acc: 0.733 (best 0.733)
In epoch 45, loss: 0.852, val acc: 0.712 (best 0.712), test acc: 0.746 (best 0.746)
In epoch 50, loss: 0.712, val acc: 0.726 (best 0.726), test acc: 0.753 (best 0.753)
In epoch 55, loss: 0.588, val acc: 0.734 (best 0.736), test acc: 0.751 (best 0

In [8]:
g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.158 (best 0.158), test acc: 0.133 (best 0.133)
In epoch 5, loss: 1.903, val acc: 0.480 (best 0.482), test acc: 0.479 (best 0.532)
In epoch 10, loss: 1.830, val acc: 0.652 (best 0.652), test acc: 0.692 (best 0.692)
In epoch 15, loss: 1.734, val acc: 0.592 (best 0.652), test acc: 0.607 (best 0.692)
In epoch 20, loss: 1.616, val acc: 0.610 (best 0.652), test acc: 0.644 (best 0.692)
In epoch 25, loss: 1.477, val acc: 0.680 (best 0.680), test acc: 0.701 (best 0.694)
In epoch 30, loss: 1.323, val acc: 0.698 (best 0.698), test acc: 0.701 (best 0.701)
In epoch 35, loss: 1.159, val acc: 0.708 (best 0.708), test acc: 0.711 (best 0.711)
In epoch 40, loss: 0.995, val acc: 0.736 (best 0.736), test acc: 0.731 (best 0.731)
In epoch 45, loss: 0.837, val acc: 0.748 (best 0.748), test acc: 0.747 (best 0.747)
In epoch 50, loss: 0.693, val acc: 0.756 (best 0.756), test acc: 0.762 (best 0.762)
In epoch 55, loss: 0.567, val acc: 0.766 (best 0.766), test acc: 0.769 (best 0